In [1]:
import json
import argparse
from src.model.api import LLM_API
from src.model.llm import LLM_local
from src.role_person_generation import generate_persona_description
from src.role_occupation_generation import generate_persona_occupation_description
from src.evaluate import evaluate_mfq30 , evaluation_pvqrr
from exp1_roleplay_nooptions import init_dataset
import re
import json
from collections import Counter
import os
API_MODEL = ["gpt-3.5-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini"]

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
#1.基本选择：选择模型，数据集，模型配置，训练方式
parser.add_argument("--dataset", type=str, default="mfq30",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--model", type=str, default="gpt-3.5-turbo",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--output_dir", type=str, default="output/exp1",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--role", type=str, default="place",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--run", type=str, default="single",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--run_results", type=str, default=None,
                    help="Path of the preprocessed dataset.")
parser.add_argument("--gpu", type=int, default=3,
                help="Random seed.")
parser.add_argument("--role_num", type=int, default=2,
                help="Random seed.")
batch_size = 1
args = parser.parse_args(args=[])
os.makedirs(args.output_dir, exist_ok=True)

with open("src/config.json", 'r') as file:
    args.config = json.load(file)



In [2]:
prompt_data = init_dataset(args.dataset)


if args.model in API_MODEL:
    model = LLM_API(args.model, api_key=args.config['openai_api_key'], base_url=args.config['base_url'])
else:
    model = LLM_local(args.model, args.gpu)
help_model =  LLM_API("gpt-3.5-turbo", api_key=args.config['openai_api_key'], base_url=args.config['base_url'])

In [3]:
if args.role=="base":
    personas = generate_persona_description(args.role_num)
elif args.role=="place":
    personas = generate_persona_occupation_description(args.role_num)


In [13]:
def extract_answer(api_model, respond, options, origin_ques):

    extract_answer_example =  f"""
    Question: {origin_ques}
    Options: A. {options[0]} B. {options[1]} C. {options[2]} D. {options[3]} E. {options[4]} F. {options[5]}
    Model Response: {respond}
    Use the above information to determine which option the model's response is more consistent with.Please output the uppercase letters of the options directly, such as A,B,C,D, and do not output anything else.
    """
    messages = []
    messages.append({"role":"user", "content": extract_answer_example})
    answer = api_model.respond(messages)
    extract_answer = answer.strip().upper()
    return extract_answer

for i, inst in enumerate(prompt_data):
    print("##############################",i,"/",len(prompt_data),"###########################################")
    answer_list = []
    for p_idx, per in enumerate(personas):
        print("----------------------------------",p_idx,"/",len(personas),"-----------------------------------")

        prompt = "Let's role-play. I will ask you a question and you must give me an answer. I want you to act as the person described below. Think from the person's perspective. \n\n"
        prompt += f"{per['description']} \n\n"
        prompt += "Use the given information to answer the question below. \n\n"
        prompt += inst['question']
        messages = []
        messages.append({"role":"user", "content": prompt})
        model_response = model.respond(messages)
        answer = extract_answer(help_model, model_response, inst['options'], inst["centerpiece"])
        break
    break

############################## 0 / 32 ###########################################
---------------------------------- 0 / 2 -----------------------------------


In [14]:
model_response

"As a translator in Egypt with a background influenced by family values and societal norms, I believe that emotional suffering should be taken into consideration when determining what is right or wrong. I understand the impact of societal prejudice and the importance of empathy towards others' emotional experiences. I believe in prioritizing family and societal interests, which includes considering the emotional well-being of individuals when making decisions. Ultimately, I believe that understanding and acknowledging emotional suffering is crucial in determining the morality of actions."

In [15]:
answer.strip().upper()

'E'